In [9]:
import pandas as pd
from datetime import datetime, timedelta

# 파일 불러오기
df = pd.read_csv("/home/youngwoo/ecg_2024-03-15.csv")


filename = "ecg_2023-07-10"
with open(f"/home/youngwoo/{filename}.csv", 'r', encoding='utf-8') as f:
    lines = f.readlines()

# "기록된 날짜"가 포함된 줄을 찾기
for line in lines:
    if '기록된 날짜' in line:
        # 시간만 추출
        datetime_str = line.strip().split(',')[-1]
        time_str = pd.to_datetime(datetime_str).time().strftime('%H:%M:%S')
        break


# 시작 시간 설정
start_time = datetime.strptime(time_str, "%H:%M:%S")

# 설정
total_rows = len(df)
start_row = 9  # 0-index 기준으로 14행
block_size = 512
num_blocks = (total_rows - start_row) // block_size

# 기존 B열 복사
timestamps = df.iloc[:, 1].copy()

# B14부터 타임스탬프 덮어쓰기
for block in range(num_blocks):
    block_start_idx = start_row + block * block_size
    block_start_time = start_time + timedelta(seconds=block)

    for i in range(block_size):
        idx = block_start_idx + i
        if idx < total_rows:
            t = block_start_time + timedelta(seconds=i / block_size)
            timestamps.iloc[idx] = t.strftime("%H:%M:%S.%f")[:-3]  # HH:MM:SS.mmm

# B열에 타임스탬프 반영
df.iloc[:, 1] = timestamps

# 저장
df.to_csv(f"{filename}_time.csv", index=False)
print("저장 완료!")


저장 완료!
